In [1]:
import pandas as pd
#math operations
import numpy as np
#machine learning
import cv2
import os
from tqdm import tqdm
import random
#for opening and loading image
from PIL import Image
#for preprocessing
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
#Doing One hot encoding as classifier has multiple classes
from random import shuffle
#MobileNetV2 model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Model, layers
from numpy import loadtxt
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf

In [2]:
model = load_model('model/cnn_model.h5')

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [3]:
img1 = "data/Tuberculosis/Tuberculosis-1.png"
img2 ="data/Tuberculosis/Tuberculosis-9.png"
img3 = "data/Normal/Normal-10.png"

In [4]:
# img_path='data/Tuberculosis/Tuberculosis-1.png'
# img = Image.open(img_path).resize((128,128)) #target_size must agree with what the trained model expects!!
# Function to load and preprocess an image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size, color_mode='grayscale')
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32') / 255.0
    img = np.repeat(img, 3, axis=-1)  # Duplicate the single channel to get 3 channels
    return img

In [5]:
# Function to predict the class of an image
def pred_image(img_path, model):
    img = load_and_preprocess_image(img_path)
    
    # Check and print the shape of the image
    print(f"Processed image shape: {img.shape}")
    
    predictions = model.predict(img)
    
    # Print the raw prediction for debugging
    print(f"Raw prediction for {img_path}: {predictions}")
    
    # Assuming binary classification with a single output representing the 'Tuberculosis' class
    predicted_class = 'Tuberculosis' if predictions[0][0] > 0.5 else 'Normal'
    confidence_percentage = predictions[0][0] * 100 if predicted_class == 'Tuberculosis' else (1 - predictions[0][0]) * 100
    
    result = [(predicted_class, confidence_percentage)]
    return result

In [6]:
pred_image(img1,model)

Processed image shape: (1, 128, 128, 3)
1/1 [==============================] - 0s 166ms/step
Raw prediction for data/Tuberculosis/Tuberculosis-1.png: [[0.963023]]


[('Tuberculosis', 96.30230069160461)]

In [7]:
pred_image(img2,model)

Processed image shape: (1, 128, 128, 3)
1/1 [==============================] - 0s 52ms/step
Raw prediction for data/Tuberculosis/Tuberculosis-9.png: [[0.99767584]]


[('Tuberculosis', 99.76758360862732)]

In [8]:
pred_image(img3,model)

Processed image shape: (1, 128, 128, 3)
1/1 [==============================] - 0s 55ms/step
Raw prediction for data/Normal/Normal-10.png: [[7.4344566e-06]]


[('Normal', 99.99925655433799)]

In [9]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)
    
    # Create a model that maps the activations of the last conv layer to the final class predictions
    classifier_input_shape = last_conv_layer.output.shape[1:]
    classifier_input = tf.keras.Input(shape=classifier_input_shape)
    
    # Manually construct the classifier model from the appropriate layers
    x = classifier_input
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(6, name="dense_2")(x)
    x = tf.keras.layers.Dropout(0.5, name="dropout_1")(x)
    x = tf.keras.layers.Dense(1, name="dense_3")(x)
    
    classifier_model = tf.keras.Model(classifier_input, x)
    
    # Compute the gradient of the top predicted class with regard to the output feature map of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    # Compute the guided gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Multiply each channel in the feature map array by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize the heatmap between 0 & 1 for visualization
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [10]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128), color_mode='grayscale')
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img.astype('float32') / 255.0
    img = np.repeat(img, 3, axis=-1)  # Duplicate the single channel to get 3 channels
    return img
    
img = load_and_preprocess_image(img1)

preds = model.predict(img)
print(preds)


1/1 [==============================] - 0s 54ms/step
[[0.963023]]


In [ ]:
# Generate class activation heatmap
last_conv_layer_name = "conv2d_5"
heatmap = make_gradcam_heatmap(img, model, last_conv_layer_name)
# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
def generate_heatmap(img_path,last_conv_layer_name):
  # Prepare image
  img_size=(224,224)

  img = Image.open(img_path).resize((224,224)) #target_size must agree with what the trained model expects!!

  # Preprocessing the image
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = img.astype('float32')/255

  # Remove last layer's softmax
  model.layers[-1].activation = None
  heatmap = make_gradcam_heatmap(img, model, last_conv_layer_name)

  return heatmap


In [ ]:
heatmap =generate_heatmap(img2,"dense_3")

In [ ]:
from IPython.display import Image as imgdisp, display
def save_and_display_gradcam(img_path, heatmap, cam_path="/content/drive/MyDrive/skin_cancer_dataset/cam.jpg", alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)


    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(imgdisp(cam_path))



In [ ]:
display(imgdisp(img1))

In [ ]:
heatmap =generate_heatmap(img1,"dense_3")
save_and_display_gradcam(img1, heatmap)

In [ ]:
heatmap =generate_heatmap(img2,"dense_3")
save_and_display_gradcam(img2, heatmap)

In [ ]:
heatmap =generate_heatmap(img3,"dense_3")
save_and_display_gradcam(img3, heatmap)